In [1]:
import tensorflow as tf
import numpy as np
import progressbar
import os.path

from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import regularizers

from tensorflow.python.keras.preprocessing import image as image_utils
from tensorflow.python.keras.applications.imagenet_utils import preprocess_input
from tensorflow.python.keras.applications.imagenet_utils import decode_predictions

# Configuration

In [2]:
src_weigths_file_name = './weights/weights_VGG16.h5'

src_train_data_path = 'F:/imagenet/cls_loc/train_224x224/'
src_val_data_path = 'F:/imagenet/cls_loc/val_224x224/'
src_test_data_path = 'F:/imagenet/cls_loc/test_224x224/'

src_meta_labels_filepath = 'F:/imagenet/meta/ILSVRC2012_mapping.txt'
src_val_labels_filepath = 'F:/imagenet/meta/ILSVRC2012_validation_ground_truth.txt'
src_test_labels_filepath = 'F:/imagenet/meta/ILSVRC2012_test_ground_truth.txt'

# Initialize input data

In [3]:
size_of_z = 3
size_of_y = 224
size_of_x = 224

number_of_classes = 1000

# Initialize model - VGG16

In [5]:
inputs = layers.Input(shape=(size_of_y, size_of_x, size_of_z))

block1_conv1  = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
block1_conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(block1_conv1)
block1_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(block1_conv2)

block2_conv1  = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(block1_pool)
block2_conv2  = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(block2_conv1)
block2_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(block2_conv2)
       
block3_conv1  = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(block2_pool)
block3_conv2  = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(block3_conv1)
block3_conv3  = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(block3_conv2)
block3_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(block3_conv3)
       
block4_conv1  = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(block3_pool)
block4_conv2  = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(block4_conv1)
block4_conv3  = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(block4_conv2)
block4_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(block4_conv3)
       
block5_conv1  = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(block4_pool)
block5_conv2  = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(block5_conv1)
block5_conv3  = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(block5_conv2)
block5_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(block5_conv3)
       
flatten = layers.Flatten()(block5_pool)

fc1 = layers.Dense(4096, activation='relu')(flatten)
fc2 = layers.Dense(4096, activation='relu')(fc1)

outputs = layers.Dense(number_of_classes, activation='softmax')(fc2)

In [6]:
model = models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.load_weights(src_weigths_file_name)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0     

In [7]:
file_mapping = open(src_meta_labels_filepath, "r")
mapping_labels_txt = file_mapping.read().split('\n')
file_mapping.close()

idx_labels = np.zeros((len(mapping_labels_txt)), dtype=np.int32) 
tag_labels = np.zeros((len(mapping_labels_txt)), dtype='|S9') 

idx_map = 0

for map_label_txt in mapping_labels_txt:
    temp = map_label_txt.split(' ')
    idx_labels[idx_map] = int(temp[0])
    tag_labels[idx_map] = temp[1]  
    idx_map += 1
    
def change_idx_to_tag(idx_labels_txt):
    expected_labels = np.zeros((len(idx_labels_txt)), dtype='|S9')
    
    idx1 = 0

    for idx_label_txt in idx_labels_txt:
        idx_label1 = int(idx_label_txt)
    
        idx2 = 0
        
        for idx_label2 in idx_labels:
            if idx_label1 == idx_label2:
                expected_labels[idx1] = tag_labels[idx2]
                break  
            idx2 += 1          
        idx1 += 1
        
    return expected_labels

In [8]:
file_val= open(src_val_labels_filepath, "r")
val_idx_labels_txt = file_val.read().split('\n')
file_val.close()

file_test= open(src_test_labels_filepath, "r")
test_idx_labels_txt = file_test.read().split('\n')
file_test.close()

val_expected_labels = change_idx_to_tag(val_idx_labels_txt)
test_expected_labels = change_idx_to_tag(test_idx_labels_txt)

val_expected_labels, test_expected_labels

(array([b'n01751748', b'n09193705', b'n02105855', ..., b'n02106166',
        b'n10148035', b'n02437616'], dtype='|S9'),
 array([b'n04008634', b'n02137549', b'n01984695', ..., b'n04461696',
        b'n07745940', b'n02437616'], dtype='|S9'))

In [ ]:
def calc_importances(output_conv_map, imp_array, eval_array):
    number_of_importances = eval_array.shape[3]

    scale_param_y = int(eval_array.shape[1] / output_conv_map.shape[0])
    scale_param_x = int(eval_array.shape[2] / output_conv_map.shape[1])

    for iZ in range(number_of_importances):
        c = 0           
       
        for iY in range(output_conv_map.shape[0]):
            for iX in range(output_conv_map.shape[1]):
                if output_conv_map[iY][iX] > 0.5:
                    k = 0.0
                        
                    for iYY in range(scale_param_y):
                        for iXX in range(scale_param_x):
                            k += eval_array[0][iY * scale_param_y + iYY][iX * scale_param_x + iXX][iZ]
                            
                    k /= number_of_importances
                    c += 1
                            
                    imp_array[iZ] += k
            
        imp_array[iZ] /= c
        
    return imp_array

def collect_informations(src_data_path, src_file_names, expected_labels, info_label): 
    count_rec_top_1 = 0
    count_rec_top_5 = 0
    
    # Begin - Initialize memory for collecting informations
    
    number_of_info_samples = 0
    
    for expected_label in expected_labels:
        if info_label == expected_label.decode('utf-8'):
            number_of_info_samples += 1

    val_info_pool_1_1 = np.zeros((number_of_info_samples, 64), dtype=np.float32)
    val_info_pool_1_2 = np.zeros((number_of_info_samples, 128), dtype=np.float32)
    val_info_pool_1_3 = np.zeros((number_of_info_samples, 256), dtype=np.float32)
    #val_info_pool_1_4 = np.zeros((512), dtype=np.float32)
    #val_info_pool_1_5 = np.zeros((512), dtype=np.float32)
    
    # End - Initialize memoryfor collecting informations
    
    idx = 0
    idx_info = 0
    
    bar = progressbar.ProgressBar(maxval=number_of_info_samples, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()

    for file_name in src_file_names:
        expected_label = expected_labels[idx].decode('utf-8')
            
        if info_label == expected_label:
            input_image = image_utils.load_img(src_data_path + src_file_names[idx])
            input_image = image_utils.img_to_array(input_image)
            input_image = np.expand_dims(input_image, axis=0)
            input_image = preprocess_input(input_image)

            output = model.predict(input_image)
            prediction = decode_predictions(output)
    
            if expected_label == prediction[0][0][0]:
                count_rec_top_1 += 1
    
            for idx_top in range(5):
                if expected_label == prediction[0][idx_top][0]:
                    count_rec_top_5 += 1
                    break
    
            # Begin - Collect informations from structure
    
            conv_1_1 = model.layers[1](input_image)
            conv_1_2 = model.layers[2](conv_1_1)
            pool_1_1 = model.layers[3](conv_1_2)
        
            conv_2_1 = model.layers[4](pool_1_1)
            conv_2_2 = model.layers[5](conv_2_1)
            pool_2_1 = model.layers[6](conv_2_2)
        
            conv_3_1 = model.layers[7](pool_2_1)
            conv_3_2 = model.layers[8](conv_3_1)
            conv_3_3 = model.layers[9](conv_3_2)
            pool_3_1 = model.layers[10](conv_3_3)
        
            conv_4_1 = model.layers[11](pool_3_1)
            conv_4_2 = model.layers[12](conv_4_1)
            conv_4_3 = model.layers[13](conv_4_2)
            pool_4_1 = model.layers[14](conv_4_3)
        
            conv_5_1 = model.layers[15](pool_4_1)
            conv_5_2 = model.layers[16](conv_5_1)
            conv_5_3 = model.layers[17](conv_5_2)
            pool_5_1 = model.layers[18](conv_5_3)
            
            output_conv_eval = pool_5_1.numpy()
            output_conv_eval = output_conv_eval / output_conv_eval.max()
            output_conv_map = np.zeros((14, 14), dtype=np.float32)
            
            for iZ in range(512):
                for iY in range(14):
                    for iX in range(14):
                        src_value = output_conv_eval[0][iY][iX][iZ]

                        if src_value > 0.5 and src_value > output_conv_map[iY][iX]:
                            output_conv_map[iY][iX] = src_value
            
            calc_importances(output_conv_map, val_info_pool_1_1[idx_info], pool_1_1.numpy())
            calc_importances(output_conv_map, val_info_pool_1_2[idx_info], pool_2_1.numpy())
            calc_importances(output_conv_map, val_info_pool_1_3[idx_info], pool_3_1.numpy())           

            # End - Collect informations from structure
            
            idx_info += 1
            
            bar.update(idx_info)
            
            #print(idx_info, '/', number_of_info_samples,
            #    '\ttop-1:', count_rec_top_1, '/', idx_info,
             #   '\ttop-5:', count_rec_top_5, '/', idx_info)
        
        idx += 1
        #break
        
    bar.finish()
    
    #print(val_info_pool_1_1)
    #print(val_info_pool_1_2)
    #print(val_info_pool_1_3)

    return number_of_info_samples, count_rec_top_1, count_rec_top_5, val_info_pool_1_1, val_info_pool_1_2, val_info_pool_1_3

# Prediction - Validation data

In [ ]:
directory_names = tf.io.gfile.listdir('F:/imagenet/cls_loc/train/')

idx = 0

for directory_name in directory_names:
    directory_name = directory_name[0:9]
    
    idx += 1

    if not os.path.isfile('X:/python_projects/classification_imagenet/importances/' + directory_name + '_info_1.bin'):
        print(directory_name, '--->', idx, '/', len(directory_names))
    
        number_of_info_samples, count_rec_top_1, count_rec_top_5, info_1, info_2, info_3 = collect_informations(src_val_data_path, tf.io.gfile.listdir(src_val_data_path), val_expected_labels, directory_name)
    
        importances_file_path_1 = 'X:/python_projects/classification_imagenet/importances/' + directory_name + '_info_1.bin'
        importances_file_path_2 = 'X:/python_projects/classification_imagenet/importances/' + directory_name + '_info_2.bin'
        importances_file_path_3 = 'X:/python_projects/classification_imagenet/importances/' + directory_name + '_info_3.bin'

        importances_file_1 = open(importances_file_path_1, 'bw')
        info_1.tofile(importances_file_1)
        importances_file_1.close()

        importances_file_2 = open(importances_file_path_2, 'bw')
        info_2.tofile(importances_file_2)
        importances_file_2.close()
    
        importances_file_3 = open(importances_file_path_3, 'bw')
        info_3.tofile(importances_file_3)
        importances_file_3.close()

In [9]:
def calc_importances(output_conv_map, imp_array, eval_array):
    number_of_importances = eval_array.shape[3]

    scale_param_y = int(eval_array.shape[1] / output_conv_map.shape[0])
    scale_param_x = int(eval_array.shape[2] / output_conv_map.shape[1])

    for iZ in range(number_of_importances):
        c = 0           
       
        for iY in range(output_conv_map.shape[0]):
            for iX in range(output_conv_map.shape[1]):
                if output_conv_map[iY][iX] > 0.5:
                    k = 0.0
                        
                    for iYY in range(scale_param_y):
                        for iXX in range(scale_param_x):
                            k += eval_array[0][iY * scale_param_y + iYY][iX * scale_param_x + iXX][iZ]
                            
                    k /= number_of_importances
                    c += 1
                            
                    imp_array[iZ] += k
            
        imp_array[iZ] /= c
        
    return imp_array

def collect_informations(src_data_path, src_file_names, number_of_info_samples): 
    # Begin - Initialize memory for collecting informations

    val_info_pool_1_1 = np.zeros((number_of_info_samples, 64), dtype=np.float32)
    val_info_pool_1_2 = np.zeros((number_of_info_samples, 128), dtype=np.float32)
    val_info_pool_1_3 = np.zeros((number_of_info_samples, 256), dtype=np.float32)
    val_info_pool_1_4 = np.zeros((number_of_info_samples, 512), dtype=np.float32)
    val_info_pool_1_5 = np.zeros((number_of_info_samples, 512), dtype=np.float32)
    
    info_dense_1 = np.zeros((number_of_info_samples, 4096), dtype=np.float32)
    info_dense_2 = np.zeros((number_of_info_samples, 4096), dtype=np.float32)
    
    # End - Initialize memoryfor collecting informations
    
    idx = 0
    
    bar = progressbar.ProgressBar(maxval=number_of_info_samples, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()

    for file_name in src_file_names:
        input_image = image_utils.load_img(src_data_path + src_file_names[idx])
        input_image = input_image.resize(size=[224, 224])
        input_image = image_utils.img_to_array(input_image)
        input_image = np.expand_dims(input_image, axis=0)
        input_image = preprocess_input(input_image)

        output = model.predict(input_image)
        prediction = decode_predictions(output)
    
        # Begin - Collect informations from structure
    
        conv_1_1 = model.layers[1](input_image)
        conv_1_2 = model.layers[2](conv_1_1)
        pool_1_1 = model.layers[3](conv_1_2)
        
        conv_2_1 = model.layers[4](pool_1_1)
        conv_2_2 = model.layers[5](conv_2_1)
        pool_2_1 = model.layers[6](conv_2_2)
        
        conv_3_1 = model.layers[7](pool_2_1)
        conv_3_2 = model.layers[8](conv_3_1)
        conv_3_3 = model.layers[9](conv_3_2)
        pool_3_1 = model.layers[10](conv_3_3)
        
        conv_4_1 = model.layers[11](pool_3_1)
        conv_4_2 = model.layers[12](conv_4_1)
        conv_4_3 = model.layers[13](conv_4_2)
        pool_4_1 = model.layers[14](conv_4_3)
        
        conv_5_1 = model.layers[15](pool_4_1)
        conv_5_2 = model.layers[16](conv_5_1)
        conv_5_3 = model.layers[17](conv_5_2)
        pool_5_1 = model.layers[18](conv_5_3)
        
        flatten = model.layers[19](pool_5_1)
        dense_1 = model.layers[20](flatten)
        dense_2 = model.layers[21](dense_1)
        
        output_conv_eval = pool_5_1.numpy()
        output_conv_eval = output_conv_eval / output_conv_eval.max()
        output_conv_map = np.zeros((7, 7), dtype=np.float32)

        for iZ in range(512):
            for iY in range(7):
                for iX in range(7):
                    src_value = output_conv_eval[0][iY][iX][iZ]

                    if src_value > 0.5 and src_value > output_conv_map[iY][iX]:
                        output_conv_map[iY][iX] = src_value
            
        calc_importances(output_conv_map, val_info_pool_1_1[idx], pool_1_1.numpy())
        calc_importances(output_conv_map, val_info_pool_1_2[idx], pool_2_1.numpy())
        calc_importances(output_conv_map, val_info_pool_1_3[idx], pool_3_1.numpy())           
        calc_importances(output_conv_map, val_info_pool_1_4[idx], pool_4_1.numpy())
        calc_importances(output_conv_map, val_info_pool_1_5[idx], pool_5_1.numpy())
        
        info_dense_1[idx] = dense_1.numpy()
        info_dense_2[idx] = dense_2.numpy()
        
        # End - Collect informations from structure
            
        bar.update(idx)
            
        idx += 1
        #break;
        
    bar.finish()
    
    #print(val_info_pool_1_1)
    #print(val_info_pool_1_2)
    #print(val_info_pool_1_3)
    #print(val_info_pool_1_4)
    #print(val_info_pool_1_5)
    
    #print(info_dense_1)
    #print(info_dense_2)
    
    return val_info_pool_1_1, val_info_pool_1_2, val_info_pool_1_3, val_info_pool_1_4, val_info_pool_1_5, info_dense_1, info_dense_2

In [10]:
from array import array

directory_names = tf.io.gfile.listdir('F:/imagenet/cls_loc/train/')

idx = 0

for directory_name in directory_names:
    directory_name = directory_name[0:9]
    
    if not os.path.isfile('X:/Databases/imagenet-1m/features/features_train/' + directory_name + '_info_1.bin'):
        aaa = 'F:/imagenet/cls_loc/train/' + directory_name + '/'
        ccc = len(tf.io.gfile.listdir(aaa))

        info_1, info_2, info_3, info_4, info_5, info_6, info_7 = collect_informations(aaa, tf.io.gfile.listdir(aaa), ccc)
        
        data_to_file_1 = info_1.reshape(ccc * 64)
        data_to_file_1 = array('f', data_to_file_1)

        data_to_file_2 = info_2.reshape(ccc * 128)
        data_to_file_2 = array('f', data_to_file_2)

        data_to_file_3 = info_3.reshape(ccc * 256)
        data_to_file_3 = array('f', data_to_file_3)

        data_to_file_4 = info_4.reshape(ccc * 512)
        data_to_file_4 = array('f', data_to_file_4)

        data_to_file_5 = info_5.reshape(ccc * 512)
        data_to_file_5 = array('f', data_to_file_5)

        data_to_file_6 = info_6.reshape(ccc * 4096)
        data_to_file_6 = array('f', data_to_file_6)

        data_to_file_7 = info_7.reshape(ccc * 4096)
        data_to_file_7 = array('f', data_to_file_7)

        outputs_file_path_1 = 'X:/Databases/imagenet-1m/features/features_train/' + directory_name + '_info_1.bin'
        outputs_file_path_2 = 'X:/Databases/imagenet-1m/features/features_train/' + directory_name + '_info_2.bin'
        outputs_file_path_3 = 'X:/Databases/imagenet-1m/features/features_train/' + directory_name + '_info_3.bin'
        outputs_file_path_4 = 'X:/Databases/imagenet-1m/features/features_train/' + directory_name + '_info_4.bin'
        outputs_file_path_5 = 'X:/Databases/imagenet-1m/features/features_train/' + directory_name + '_info_5.bin'
        outputs_file_path_6 = 'X:/Databases/imagenet-1m/features/features_train/' + directory_name + '_info_6.bin'
        outputs_file_path_7 = 'X:/Databases/imagenet-1m/features/features_train/' + directory_name + '_info_7.bin'

        outputs_file_1 = open(outputs_file_path_1, 'bw')
        info_1.tofile(outputs_file_1)
        outputs_file_1.close()

        outputs_file_2 = open(outputs_file_path_2, 'bw')
        info_2.tofile(outputs_file_2)
        outputs_file_2.close()

        outputs_file_3 = open(outputs_file_path_3, 'bw')
        info_3.tofile(outputs_file_3)
        outputs_file_3.close()

        outputs_file_4 = open(outputs_file_path_4, 'bw')
        info_4.tofile(outputs_file_4)
        outputs_file_4.close()

        outputs_file_5 = open(outputs_file_path_5, 'bw')
        info_5.tofile(outputs_file_5)
        outputs_file_5.close()

        outputs_file_6 = open(outputs_file_path_6, 'bw')
        info_6.tofile(outputs_file_6)
        outputs_file_6.close()

        outputs_file_7 = open(outputs_file_path_7, 'bw')
        info_7.tofile(outputs_file_7)
        outputs_file_7.close()

    idx += 1
    
    print(directory_name, '--->', idx, '/', len(directory_names))   

n01440764 ---> 1 / 1000
n01443537 ---> 2 / 1000
n01484850 ---> 3 / 1000
n01491361 ---> 4 / 1000
n01494475 ---> 5 / 1000
n01496331 ---> 6 / 1000
n01498041 ---> 7 / 1000
n01514668 ---> 8 / 1000
n01514859 ---> 9 / 1000
n01518878 ---> 10 / 1000
n01530575 ---> 11 / 1000
n01531178 ---> 12 / 1000
n01532829 ---> 13 / 1000
n01534433 ---> 14 / 1000
n01537544 ---> 15 / 1000
n01558993 ---> 16 / 1000
n01560419 ---> 17 / 1000
n01580077 ---> 18 / 1000
n01582220 ---> 19 / 1000
n01592084 ---> 20 / 1000
n01601694 ---> 21 / 1000
n01608432 ---> 22 / 1000
n01614925 ---> 23 / 1000
n01616318 ---> 24 / 1000
n01622779 ---> 25 / 1000
n01629819 ---> 26 / 1000
n01630670 ---> 27 / 1000
n01631663 ---> 28 / 1000
n01632458 ---> 29 / 1000
n01632777 ---> 30 / 1000
n01641577 ---> 31 / 1000
n01644373 ---> 32 / 1000
n01644900 ---> 33 / 1000
n01664065 ---> 34 / 1000
n01665541 ---> 35 / 1000
n01667114 ---> 36 / 1000
n01667778 ---> 37 / 1000
n01669191 ---> 38 / 1000
n01675722 ---> 39 / 1000
n01677366 ---> 40 / 1000
n01682714

n02276258 ---> 322 / 1000
n02277742 ---> 323 / 1000
n02279972 ---> 324 / 1000
n02280649 ---> 325 / 1000
n02281406 ---> 326 / 1000
n02281787 ---> 327 / 1000
n02317335 ---> 328 / 1000
n02319095 ---> 329 / 1000
n02321529 ---> 330 / 1000
n02325366 ---> 331 / 1000
n02326432 ---> 332 / 1000
n02328150 ---> 333 / 1000
n02342885 ---> 334 / 1000
n02346627 ---> 335 / 1000
n02356798 ---> 336 / 1000
n02361337 ---> 337 / 1000
n02363005 ---> 338 / 1000
n02364673 ---> 339 / 1000
n02389026 ---> 340 / 1000
n02391049 ---> 341 / 1000
n02395406 ---> 342 / 1000
n02396427 ---> 343 / 1000
n02397096 ---> 344 / 1000
n02398521 ---> 345 / 1000
n02403003 ---> 346 / 1000
n02408429 ---> 347 / 1000
n02410509 ---> 348 / 1000
n02412080 ---> 349 / 1000
n02415577 ---> 350 / 1000
n02417914 ---> 351 / 1000
n02422106 ---> 352 / 1000
n02422699 ---> 353 / 1000
n02423022 ---> 354 / 1000
n02437312 ---> 355 / 1000
n02437616 ---> 356 / 1000
n02441942 ---> 357 / 1000
n02442845 ---> 358 / 1000
n02443114 ---> 359 / 1000
n02443484 --

n03871628 ---> 693 / 1000
n03873416 ---> 694 / 1000
n03874293 ---> 695 / 1000
n03874599 ---> 696 / 1000
n03876231 ---> 697 / 1000
n03877472 ---> 698 / 1000
n03877845 ---> 699 / 1000
n03884397 ---> 700 / 1000
n03887697 ---> 701 / 1000
n03888257 ---> 702 / 1000
n03888605 ---> 703 / 1000
n03891251 ---> 704 / 1000
n03891332 ---> 705 / 1000
n03895866 ---> 706 / 1000
n03899768 ---> 707 / 1000
n03902125 ---> 708 / 1000
n03903868 ---> 709 / 1000
n03908618 ---> 710 / 1000
n03908714 ---> 711 / 1000
n03916031 ---> 712 / 1000
n03920288 ---> 713 / 1000
n03924679 ---> 714 / 1000
n03929660 ---> 715 / 1000
n03929855 ---> 716 / 1000
n03930313 ---> 717 / 1000
n03930630 ---> 718 / 1000
n03933933 ---> 719 / 1000
n03935335 ---> 720 / 1000
n03937543 ---> 721 / 1000
n03938244 ---> 722 / 1000
n03942813 ---> 723 / 1000
n03944341 ---> 724 / 1000
n03947888 ---> 725 / 1000
n03950228 ---> 726 / 1000
n03954731 ---> 727 / 1000
n03956157 ---> 728 / 1000
n03958227 ---> 729 / 1000
n03961711 ---> 730 / 1000
n03967562 --

 ---> 871 / 1000
n04483307 ---> 872 / 1000
n04485082 ---> 873 / 1000
n04486054 ---> 874 / 1000
n04487081 ---> 875 / 1000
n04487394 ---> 876 / 1000
n04493381 ---> 877 / 1000
n04501370 ---> 878 / 1000
n04505470 ---> 879 / 1000
n04507155 ---> 880 / 1000
n04509417 ---> 881 / 1000
n04515003 ---> 882 / 1000
n04517823 ---> 883 / 1000
n04522168 ---> 884 / 1000
n04523525 ---> 885 / 1000
n04525038 ---> 886 / 1000
n04525305 ---> 887 / 1000
n04532106 ---> 888 / 1000
n04532670 ---> 889 / 1000
n04536866 ---> 890 / 1000
n04540053 ---> 891 / 1000
n04542943 ---> 892 / 1000
n04548280 ---> 893 / 1000
n04548362 ---> 894 / 1000
n04550184 ---> 895 / 1000
n04552348 ---> 896 / 1000
n04553703 ---> 897 / 1000
n04554684 ---> 898 / 1000
n04557648 ---> 899 / 1000
n04560804 ---> 900 / 1000
n04562935 ---> 901 / 1000
n04579145 ---> 902 / 1000
n04579432 ---> 903 / 1000
n04584207 ---> 904 / 1000
n04589890 ---> 905 / 1000
n04590129 ---> 906 / 1000
n04591157 ---> 907 / 1000
n04591713 ---> 908 / 1000
n04592741 ---> 909 / 

[========================================================================] 100%


...

n13054560 ---> 998 / 1000


[========================================================================] 100%


n13133613 ---> 999 / 1000


[========================================================================] 100%


n15075141 ---> 1000 / 1000


In [17]:
def calc_importances(output_conv_map, imp_array, eval_array):
    number_of_importances = eval_array.shape[3]

    scale_param_y = int(eval_array.shape[1] / output_conv_map.shape[0])
    scale_param_x = int(eval_array.shape[2] / output_conv_map.shape[1])

    for iZ in range(number_of_importances):
        c = 0           
       
        for iY in range(output_conv_map.shape[0]):
            for iX in range(output_conv_map.shape[1]):
                if output_conv_map[iY][iX] > 0.5:
                    k = 0.0
                        
                    for iYY in range(scale_param_y):
                        for iXX in range(scale_param_x):
                            k += eval_array[0][iY * scale_param_y + iYY][iX * scale_param_x + iXX][iZ]
                            
                    k /= number_of_importances
                    c += 1
                            
                    imp_array[iZ] += k
            
        imp_array[iZ] /= c
        
    return imp_array

def collect_informations(src_data_path, src_file_names, number_of_info_samples): 
    # Begin - Initialize memory for collecting informations

    val_info_pool_1_1 = np.zeros((number_of_info_samples, 64), dtype=np.float32)
    val_info_pool_1_2 = np.zeros((number_of_info_samples, 128), dtype=np.float32)
    val_info_pool_1_3 = np.zeros((number_of_info_samples, 256), dtype=np.float32)
    val_info_pool_1_4 = np.zeros((number_of_info_samples, 512), dtype=np.float32)
    val_info_pool_1_5 = np.zeros((number_of_info_samples, 512), dtype=np.float32)
    
    info_dense_1 = np.zeros((number_of_info_samples, 4096), dtype=np.float32)
    info_dense_2 = np.zeros((number_of_info_samples, 4096), dtype=np.float32)
    
    # End - Initialize memoryfor collecting informations
    
    idx = 0
    
    for file_name in src_file_names:
        input_image = image_utils.load_img(src_data_path + src_file_names[idx])
        input_image = image_utils.img_to_array(input_image)
        input_image = np.expand_dims(input_image, axis=0)
        input_image = preprocess_input(input_image)

        output = model.predict(input_image)
        prediction = decode_predictions(output)
    
        # Begin - Collect informations from structure
    
        conv_1_1 = model.layers[1](input_image)
        conv_1_2 = model.layers[2](conv_1_1)
        pool_1_1 = model.layers[3](conv_1_2)
        
        conv_2_1 = model.layers[4](pool_1_1)
        conv_2_2 = model.layers[5](conv_2_1)
        pool_2_1 = model.layers[6](conv_2_2)
        
        conv_3_1 = model.layers[7](pool_2_1)
        conv_3_2 = model.layers[8](conv_3_1)
        conv_3_3 = model.layers[9](conv_3_2)
        pool_3_1 = model.layers[10](conv_3_3)
        
        conv_4_1 = model.layers[11](pool_3_1)
        conv_4_2 = model.layers[12](conv_4_1)
        conv_4_3 = model.layers[13](conv_4_2)
        pool_4_1 = model.layers[14](conv_4_3)
        
        conv_5_1 = model.layers[15](pool_4_1)
        conv_5_2 = model.layers[16](conv_5_1)
        conv_5_3 = model.layers[17](conv_5_2)
        pool_5_1 = model.layers[18](conv_5_3)
        
        flatten = model.layers[19](pool_5_1)
        dense_1 = model.layers[20](flatten)
        dense_2 = model.layers[21](dense_1)
        
        output_conv_eval = pool_5_1.numpy()
        output_conv_eval = output_conv_eval / output_conv_eval.max()
        output_conv_map = np.zeros((7, 7), dtype=np.float32)

        for iZ in range(512):
            for iY in range(7):
                for iX in range(7):
                    src_value = output_conv_eval[0][iY][iX][iZ]

                    if src_value > 0.5 and src_value > output_conv_map[iY][iX]:
                        output_conv_map[iY][iX] = src_value
            
        calc_importances(output_conv_map, val_info_pool_1_1[idx], pool_1_1.numpy())
        calc_importances(output_conv_map, val_info_pool_1_2[idx], pool_2_1.numpy())
        calc_importances(output_conv_map, val_info_pool_1_3[idx], pool_3_1.numpy())           
        calc_importances(output_conv_map, val_info_pool_1_4[idx], pool_4_1.numpy())
        calc_importances(output_conv_map, val_info_pool_1_5[idx], pool_5_1.numpy())
        
        info_dense_1[idx] = dense_1.numpy()
        info_dense_2[idx] = dense_2.numpy()
        
        # End - Collect informations from structure
            
        idx += 1
        
        print(idx, '/', len(src_file_names))
        #break;
    
    #print(val_info_pool_1_1)
    #print(val_info_pool_1_2)
    #print(val_info_pool_1_3)
    #print(val_info_pool_1_4)
    #print(val_info_pool_1_5)
    
    #print(info_dense_1)
    #print(info_dense_2)
    
    return val_info_pool_1_1, val_info_pool_1_2, val_info_pool_1_3, val_info_pool_1_4, val_info_pool_1_5, info_dense_1, info_dense_2

In [18]:
from array import array

val_244 = tf.io.gfile.listdir('F:/imagenet/cls_loc/val_224x224/')
val_len = len(val_244)

info_1, info_2, info_3, info_4, info_5, info_6, info_7 = collect_informations('F:/imagenet/cls_loc/val_224x224/', val_244, val_len)
        
data_to_file_1 = info_1.reshape(ccc * 64)
data_to_file_1 = array('f', data_to_file_1)

data_to_file_2 = info_2.reshape(ccc * 128)
data_to_file_2 = array('f', data_to_file_2)

data_to_file_3 = info_3.reshape(ccc * 256)
data_to_file_3 = array('f', data_to_file_3)

data_to_file_4 = info_4.reshape(ccc * 512)
data_to_file_4 = array('f', data_to_file_4)

data_to_file_5 = info_5.reshape(ccc * 512)
data_to_file_5 = array('f', data_to_file_5)

data_to_file_6 = info_6.reshape(ccc * 4096)
data_to_file_6 = array('f', data_to_file_6)

data_to_file_7 = info_7.reshape(ccc * 4096)
data_to_file_7 = array('f', data_to_file_7)

outputs_file_path_1 = 'X:/Databases/imagenet-1m/features/features_query_2/info_1.bin'
outputs_file_path_2 = 'X:/Databases/imagenet-1m/features/features_query_2/info_2.bin'
outputs_file_path_3 = 'X:/Databases/imagenet-1m/features/features_query_2/info_3.bin'
outputs_file_path_4 = 'X:/Databases/imagenet-1m/features/features_query_2/info_4.bin'
outputs_file_path_5 = 'X:/Databases/imagenet-1m/features/features_query_2/info_5.bin'
outputs_file_path_6 = 'X:/Databases/imagenet-1m/features/features_query_2/info_6.bin'
outputs_file_path_7 = 'X:/Databases/imagenet-1m/features/features_query_2/info_7.bin'

outputs_file_1 = open(outputs_file_path_1, 'bw')
info_1.tofile(outputs_file_1)
outputs_file_1.close()

outputs_file_2 = open(outputs_file_path_2, 'bw')
info_2.tofile(outputs_file_2)
outputs_file_2.close()

outputs_file_3 = open(outputs_file_path_3, 'bw')
info_3.tofile(outputs_file_3)
outputs_file_3.close()

outputs_file_4 = open(outputs_file_path_4, 'bw')
info_4.tofile(outputs_file_4)
outputs_file_4.close()

outputs_file_5 = open(outputs_file_path_5, 'bw')
info_5.tofile(outputs_file_5)
outputs_file_5.close()

outputs_file_6 = open(outputs_file_path_6, 'bw')
info_6.tofile(outputs_file_6)
outputs_file_6.close()

outputs_file_7 = open(outputs_file_path_7, 'bw')
info_7.tofile(outputs_file_7)
outputs_file_7.close()

1 / 50000
2 / 50000
3 / 50000
4 / 50000
5 / 50000
6 / 50000
7 / 50000
8 / 50000
9 / 50000
10 / 50000
11 / 50000
12 / 50000
13 / 50000
14 / 50000
15 / 50000
16 / 50000
17 / 50000
18 / 50000
19 / 50000
20 / 50000
21 / 50000
22 / 50000
23 / 50000
24 / 50000
25 / 50000
26 / 50000
27 / 50000
28 / 50000
29 / 50000
30 / 50000
31 / 50000
32 / 50000
33 / 50000
34 / 50000
35 / 50000
36 / 50000
37 / 50000
38 / 50000
39 / 50000
40 / 50000
41 / 50000
42 / 50000
43 / 50000
44 / 50000
45 / 50000
46 / 50000
47 / 50000
48 / 50000
49 / 50000
50 / 50000
51 / 50000
52 / 50000
53 / 50000
54 / 50000
55 / 50000
56 / 50000
57 / 50000
58 / 50000
59 / 50000
60 / 50000
61 / 50000
62 / 50000
63 / 50000
64 / 50000
65 / 50000
66 / 50000
67 / 50000
68 / 50000
69 / 50000
70 / 50000
71 / 50000
72 / 50000
73 / 50000
74 / 50000
75 / 50000
76 / 50000
77 / 50000
78 / 50000
79 / 50000
80 / 50000
81 / 50000
82 / 50000
83 / 50000
84 / 50000
85 / 50000
86 / 50000
87 / 50000
88 / 50000
89 / 50000
90 / 50000
91 / 50000
92 / 500

693 / 50000
694 / 50000
695 / 50000
696 / 50000
697 / 50000
698 / 50000
699 / 50000
700 / 50000
701 / 50000
702 / 50000
703 / 50000
704 / 50000
705 / 50000
706 / 50000
707 / 50000
708 / 50000
709 / 50000
710 / 50000
711 / 50000
712 / 50000
713 / 50000
714 / 50000
715 / 50000
716 / 50000
717 / 50000
718 / 50000
719 / 50000
720 / 50000
721 / 50000
722 / 50000
723 / 50000
724 / 50000
725 / 50000
726 / 50000
727 / 50000
728 / 50000
729 / 50000
730 / 50000
731 / 50000
732 / 50000
733 / 50000
734 / 50000
735 / 50000
736 / 50000
737 / 50000
738 / 50000
739 / 50000
740 / 50000
741 / 50000
742 / 50000
743 / 50000
744 / 50000
745 / 50000
746 / 50000
747 / 50000
748 / 50000
749 / 50000
750 / 50000
751 / 50000
752 / 50000
753 / 50000
754 / 50000
755 / 50000
756 / 50000
757 / 50000
758 / 50000
759 / 50000
760 / 50000
761 / 50000
762 / 50000
763 / 50000
764 / 50000
765 / 50000
766 / 50000
767 / 50000
768 / 50000
769 / 50000
770 / 50000
771 / 50000
772 / 50000
773 / 50000
774 / 50000
775 / 50000
776 

1347 / 50000
1348 / 50000
1349 / 50000
1350 / 50000
1351 / 50000
1352 / 50000
1353 / 50000
1354 / 50000
1355 / 50000
1356 / 50000
1357 / 50000
1358 / 50000
1359 / 50000
1360 / 50000
1361 / 50000
1362 / 50000
1363 / 50000
1364 / 50000
1365 / 50000
1366 / 50000
1367 / 50000
1368 / 50000
1369 / 50000
1370 / 50000
1371 / 50000
1372 / 50000
1373 / 50000
1374 / 50000
1375 / 50000
1376 / 50000
1377 / 50000
1378 / 50000
1379 / 50000
1380 / 50000
1381 / 50000
1382 / 50000
1383 / 50000
1384 / 50000
1385 / 50000
1386 / 50000
1387 / 50000
1388 / 50000
1389 / 50000
1390 / 50000
1391 / 50000
1392 / 50000
1393 / 50000
1394 / 50000
1395 / 50000
1396 / 50000
1397 / 50000
1398 / 50000
1399 / 50000
1400 / 50000
1401 / 50000
1402 / 50000
1403 / 50000
1404 / 50000
1405 / 50000
1406 / 50000
1407 / 50000
1408 / 50000
1409 / 50000
1410 / 50000
1411 / 50000
1412 / 50000
1413 / 50000
1414 / 50000
1415 / 50000
1416 / 50000
1417 / 50000
1418 / 50000
1419 / 50000
1420 / 50000
1421 / 50000
1422 / 50000
1423 / 50000

1978 / 50000
1979 / 50000
1980 / 50000
1981 / 50000
1982 / 50000
1983 / 50000
1984 / 50000
1985 / 50000
1986 / 50000
1987 / 50000
1988 / 50000
1989 / 50000
1990 / 50000
1991 / 50000
1992 / 50000
1993 / 50000
1994 / 50000
1995 / 50000
1996 / 50000
1997 / 50000
1998 / 50000
1999 / 50000
2000 / 50000
2001 / 50000
2002 / 50000
2003 / 50000
2004 / 50000
2005 / 50000
2006 / 50000
2007 / 50000
2008 / 50000
2009 / 50000
2010 / 50000
2011 / 50000
2012 / 50000
2013 / 50000
2014 / 50000
2015 / 50000
2016 / 50000
2017 / 50000
2018 / 50000
2019 / 50000
2020 / 50000
2021 / 50000
2022 / 50000
2023 / 50000
2024 / 50000
2025 / 50000
2026 / 50000
2027 / 50000
2028 / 50000
2029 / 50000
2030 / 50000
2031 / 50000
2032 / 50000
2033 / 50000
2034 / 50000
2035 / 50000
2036 / 50000
2037 / 50000
2038 / 50000
2039 / 50000
2040 / 50000
2041 / 50000
2042 / 50000
2043 / 50000
2044 / 50000
2045 / 50000
2046 / 50000
2047 / 50000
2048 / 50000
2049 / 50000
2050 / 50000
2051 / 50000
2052 / 50000
2053 / 50000
2054 / 50000

2609 / 50000
2610 / 50000
2611 / 50000
2612 / 50000
2613 / 50000
2614 / 50000
2615 / 50000
2616 / 50000
2617 / 50000
2618 / 50000
2619 / 50000
2620 / 50000
2621 / 50000
2622 / 50000
2623 / 50000
2624 / 50000
2625 / 50000
2626 / 50000
2627 / 50000
2628 / 50000
2629 / 50000
2630 / 50000
2631 / 50000
2632 / 50000
2633 / 50000
2634 / 50000
2635 / 50000
2636 / 50000
2637 / 50000
2638 / 50000
2639 / 50000
2640 / 50000
2641 / 50000
2642 / 50000
2643 / 50000
2644 / 50000
2645 / 50000
2646 / 50000
2647 / 50000
2648 / 50000
2649 / 50000
2650 / 50000
2651 / 50000
2652 / 50000
2653 / 50000
2654 / 50000
2655 / 50000
2656 / 50000
2657 / 50000
2658 / 50000
2659 / 50000
2660 / 50000
2661 / 50000
2662 / 50000
2663 / 50000
2664 / 50000
2665 / 50000
2666 / 50000
2667 / 50000
2668 / 50000
2669 / 50000
2670 / 50000
2671 / 50000
2672 / 50000
2673 / 50000
2674 / 50000
2675 / 50000
2676 / 50000
2677 / 50000
2678 / 50000
2679 / 50000
2680 / 50000
2681 / 50000
2682 / 50000
2683 / 50000
2684 / 50000
2685 / 50000

...

47086 / 50000
47087 / 50000
47088 / 50000
47089 / 50000
47090 / 50000
47091 / 50000
47092 / 50000
47093 / 50000
47094 / 50000
47095 / 50000
47096 / 50000
47097 / 50000
47098 / 50000
47099 / 50000
47100 / 50000
47101 / 50000
47102 / 50000
47103 / 50000
47104 / 50000
47105 / 50000
47106 / 50000
47107 / 50000
47108 / 50000
47109 / 50000
47110 / 50000
47111 / 50000
47112 / 50000
47113 / 50000
47114 / 50000
47115 / 50000
47116 / 50000
47117 / 50000
47118 / 50000
47119 / 50000
47120 / 50000
47121 / 50000
47122 / 50000
47123 / 50000
47124 / 50000
47125 / 50000
47126 / 50000
47127 / 50000
47128 / 50000
47129 / 50000
47130 / 50000
47131 / 50000
47132 / 50000
47133 / 50000
47134 / 50000
47135 / 50000
47136 / 50000
47137 / 50000
47138 / 50000
47139 / 50000
47140 / 50000
47141 / 50000
47142 / 50000
47143 / 50000
47144 / 50000
47145 / 50000
47146 / 50000
47147 / 50000
47148 / 50000
47149 / 50000
47150 / 50000
47151 / 50000
47152 / 50000
47153 / 50000
47154 / 50000
47155 / 50000
47156 / 50000
47157 

47672 / 50000
47673 / 50000
47674 / 50000
47675 / 50000
47676 / 50000
47677 / 50000
47678 / 50000
47679 / 50000
47680 / 50000
47681 / 50000
47682 / 50000
47683 / 50000
47684 / 50000
47685 / 50000
47686 / 50000
47687 / 50000
47688 / 50000
47689 / 50000
47690 / 50000
47691 / 50000
47692 / 50000
47693 / 50000
47694 / 50000
47695 / 50000
47696 / 50000
47697 / 50000
47698 / 50000
47699 / 50000
47700 / 50000
47701 / 50000
47702 / 50000
47703 / 50000
47704 / 50000
47705 / 50000
47706 / 50000
47707 / 50000
47708 / 50000
47709 / 50000
47710 / 50000
47711 / 50000
47712 / 50000
47713 / 50000
47714 / 50000
47715 / 50000
47716 / 50000
47717 / 50000
47718 / 50000
47719 / 50000
47720 / 50000
47721 / 50000
47722 / 50000
47723 / 50000
47724 / 50000
47725 / 50000
47726 / 50000
47727 / 50000
47728 / 50000
47729 / 50000
47730 / 50000
47731 / 50000
47732 / 50000
47733 / 50000
47734 / 50000
47735 / 50000
47736 / 50000
47737 / 50000
47738 / 50000
47739 / 50000
47740 / 50000
47741 / 50000
47742 / 50000
47743 

48258 / 50000
48259 / 50000
48260 / 50000
48261 / 50000
48262 / 50000
48263 / 50000
48264 / 50000
48265 / 50000
48266 / 50000
48267 / 50000
48268 / 50000
48269 / 50000
48270 / 50000
48271 / 50000
48272 / 50000
48273 / 50000
48274 / 50000
48275 / 50000
48276 / 50000
48277 / 50000
48278 / 50000
48279 / 50000
48280 / 50000
48281 / 50000
48282 / 50000
48283 / 50000
48284 / 50000
48285 / 50000
48286 / 50000
48287 / 50000
48288 / 50000
48289 / 50000
48290 / 50000
48291 / 50000
48292 / 50000
48293 / 50000
48294 / 50000
48295 / 50000
48296 / 50000
48297 / 50000
48298 / 50000
48299 / 50000
48300 / 50000
48301 / 50000
48302 / 50000
48303 / 50000
48304 / 50000
48305 / 50000
48306 / 50000
48307 / 50000
48308 / 50000
48309 / 50000
48310 / 50000
48311 / 50000
48312 / 50000
48313 / 50000
48314 / 50000
48315 / 50000
48316 / 50000
48317 / 50000
48318 / 50000
48319 / 50000
48320 / 50000
48321 / 50000
48322 / 50000
48323 / 50000
48324 / 50000
48325 / 50000
48326 / 50000
48327 / 50000
48328 / 50000
48329 

48844 / 50000
48845 / 50000
48846 / 50000
48847 / 50000
48848 / 50000
48849 / 50000
48850 / 50000
48851 / 50000
48852 / 50000
48853 / 50000
48854 / 50000
48855 / 50000
48856 / 50000
48857 / 50000
48858 / 50000
48859 / 50000
48860 / 50000
48861 / 50000
48862 / 50000
48863 / 50000
48864 / 50000
48865 / 50000
48866 / 50000
48867 / 50000
48868 / 50000
48869 / 50000
48870 / 50000
48871 / 50000
48872 / 50000
48873 / 50000
48874 / 50000
48875 / 50000
48876 / 50000
48877 / 50000
48878 / 50000
48879 / 50000
48880 / 50000
48881 / 50000
48882 / 50000
48883 / 50000
48884 / 50000
48885 / 50000
48886 / 50000
48887 / 50000
48888 / 50000
48889 / 50000
48890 / 50000
48891 / 50000
48892 / 50000
48893 / 50000
48894 / 50000
48895 / 50000
48896 / 50000
48897 / 50000
48898 / 50000
48899 / 50000
48900 / 50000
48901 / 50000
48902 / 50000
48903 / 50000
48904 / 50000
48905 / 50000
48906 / 50000
48907 / 50000
48908 / 50000
48909 / 50000
48910 / 50000
48911 / 50000
48912 / 50000
48913 / 50000
48914 / 50000
48915 

49430 / 50000
49431 / 50000
49432 / 50000
49433 / 50000
49434 / 50000
49435 / 50000
49436 / 50000
49437 / 50000
49438 / 50000
49439 / 50000
49440 / 50000
49441 / 50000
49442 / 50000
49443 / 50000
49444 / 50000
49445 / 50000
49446 / 50000
49447 / 50000
49448 / 50000
49449 / 50000
49450 / 50000
49451 / 50000
49452 / 50000
49453 / 50000
49454 / 50000
49455 / 50000
49456 / 50000
49457 / 50000
49458 / 50000
49459 / 50000
49460 / 50000
49461 / 50000
49462 / 50000
49463 / 50000
49464 / 50000
49465 / 50000
49466 / 50000
49467 / 50000
49468 / 50000
49469 / 50000
49470 / 50000
49471 / 50000
49472 / 50000
49473 / 50000
49474 / 50000
49475 / 50000
49476 / 50000
49477 / 50000
49478 / 50000
49479 / 50000
49480 / 50000
49481 / 50000
49482 / 50000
49483 / 50000
49484 / 50000
49485 / 50000
49486 / 50000
49487 / 50000
49488 / 50000
49489 / 50000
49490 / 50000
49491 / 50000
49492 / 50000
49493 / 50000
49494 / 50000
49495 / 50000
49496 / 50000
49497 / 50000
49498 / 50000
49499 / 50000
49500 / 50000
49501 

In [ ]:
#files = tf.io.gfile.listdir('F:/imagenet/cls_loc/test/')

#for file in files:
#    image = image_utils.load_img('F:/imagenet/cls_loc/test/' + file, target_size=(224, 224))
#    image = image_utils.img_to_array(image)
#    image_utils.save_img('F:/imagenet/cls_loc/test_224x224/' + file, image)